In [ ]:
import asyncio
import websockets
import threading
import time
import sys

# Storage lists
time_values = []
acc_x_values = []
acc_y_values = []
acc_z_values = []
mag_x_values = []
mag_y_values = []
mag_z_values = []
roll_values = []
pitch_values = []
yaw_values = []
async def receive_data():
    uri = "ws://192.168.4.1:81/"
    print(f"Attempting to connect to {uri}...")

    try:
        async with websockets.connect(uri, ping_interval=None) as websocket:
            print("✅ Connected to WebSocket server!")

            while True:
                print("Waiting for data...")
                try:
                    data = await websocket.recv()
                    print(f"Raw data received: {data}")

                    values = data.split(",")
                    print(f"Split values: {values}")

                    # Check if there are exactly 7 values
                    if len(values) != 10:
                        print(f"⚠️ Incorrect number of values! Expected 10, got {len(values)}")
                        continue  # Skip this iteration if data is not complete

                    # Parse the values and append to the respective lists
                    try:
                        roll, pitch, yaw, acc_x, acc_y, acc_z, mag_x, mag_y, mag_z, timestamp = map(float, values)

                        # Append data to the lists
                        time_values.append(timestamp)
                        acc_x_values.append(acc_x)
                        acc_y_values.append(acc_y)
                        acc_z_values.append(acc_z)
                        mag_x_values.append(mag_x)
                        mag_y_values.append(mag_y)
                        mag_z_values.append(mag_z)
                        roll_values.append(roll)
                        pitch_values.append(pitch)
                        yaw_values.append(yaw)

                        print(f"✅ Valid Data Stored: Time: {timestamp}, Roll: {roll}, Pitch: {pitch}, Yaw: {yaw}, Acc X: {acc_x}, Acc Y: {acc_y}, Acc Z: {acc_z}, Mag X: {mag_x}, Mag Y: {mag_y}, Mag Z: {mag_z}")

                    except ValueError as e:
                        print(f"❌ Data Parsing Error: {data} - {e}")

                except Exception as e:
                    print(f"❌ Error receiving data: {e}")
                    break

    except websockets.exceptions.ConnectionClosed as e:
        print(f"⚠️ WebSocket connection closed. Reason: {e}")
        print_collected_data()
    except Exception as e:
        print(f"❌ Connection error: {e}")

def print_collected_data():
    """Function to print previously collected data after disconnect."""
    print("=== Data Collected ===")
    print(f"Total data points: {len(time_values)}")
    if len(time_values) > 0:
        print("Data samples:")
        for i in range(min(5, len(time_values))):
            print(f"Time: {time_values[i]}, Acc X: {acc_x_values[i]}, Acc Y: {acc_y_values[i]}, Acc Z: {acc_z_values[i]}, Mag X: {mag_x_values[i]}, Mag Y: {mag_y_values[i]}, Mag Z: {mag_z_values[i]}")

# Function to start the event loop in a separate thread
def start_async_loop():
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    try:
        loop.run_until_complete(receive_data())
    except Exception as e:
        print(f"❌ Async loop error: {e}")

# Start the WebSocket listener in a separate thread
thread = threading.Thread(target=start_async_loop)
thread.daemon = True  # Make the thread exit when the main program exits
thread.start()

print("WebSocket client started. Waiting for data...")
try:
    # Keep the main thread alive
    while thread.is_alive():
        time.sleep(1)
except KeyboardInterrupt:
    print("Program terminated by user")

In [ ]:
ymin = min(acc_y_values)
low_index = acc_y_values.index(min(acc_x_values))-10

impact_vector = np.array([acc_x_values[low_index], acc_y_values[low_index], acc_z_values[low_index]])
print('Impact vector: ', impact_vector)
face_to_path = ((acc_z_values[low_index]/acc_x_values[low_index])*10/1.5)
print('Face to Path: ', face_to_path)
if face_to_path > 0:
    print('Fade')
if face_to_path < 0:
    print('Draw')

face_angle = roll_values[low_index]
attack_angle = yaw_values[low_index]

# def most_stable_section(data, window_size):
#     min_variance = float('inf')
#     best_start_index = 0
#
#     for i in range(len(data) - window_size + 1):
#         window = data[i:i + window_size]
#         variance = np.var(window)  # Calculate variance
#
#         if variance < min_variance:
#             min_variance = variance
#             best_start_index = i
#
#     return best_start_index, data[best_start_index:best_start_index + window_size]
#
# window = 100
# index, stable_section = most_stable_section(acc_y_values, window)
# plane_y = sum(acc_y_values[index:index+window])/len(acc_y_values[index:index+window])
# plane_x = sum(acc_x_values[index:index+window])/len(acc_x_values[index:index+window])
# plane_z = sum(acc_z_values[index:index+window])/len(acc_z_values[index:index+window])
# print(plane_x, plane_y, plane_z)
# if -0.3 < plane_x < 0.3:
#     plane_x = 0
#
# gravity_vector = np.array([plane_x, plane_y, plane_z])
# gravity_vector = gravity_vector / np.linalg.norm(gravity_vector)
# def vector_projection(v, u):
#     """Projects vector v onto vector u."""
#     return (np.dot(v, u) / np.dot(u, u)) * u
#
# arbitrary_vector = np.array([0, 1, 0])
# # First orthogonal axis
# orthogonal_1 = arbitrary_vector - vector_projection(arbitrary_vector, gravity_vector)
# orthogonal_1 /= np.linalg.norm(orthogonal_1)  # Normalize
#
# # Second orthogonal axis
# orthogonal_2 = np.cross(gravity_vector, orthogonal_1)
# orthogonal_2 /= np.linalg.norm(orthogonal_2)  # Normalize
#
# # Project the vector onto the two new axes
# proj_onto_1 = vector_projection(impact_vector, orthogonal_1)
# proj_onto_2 = vector_projection(impact_vector, orthogonal_2)
#
# # Find the projection onto the plane (sum of both projections)
# proj_onto_plane = proj_onto_1 + proj_onto_2


accel = []
vel = [0]
for i, j, k in zip(acc_x_values, acc_y_values, acc_z_values):
    accel.append(np.sqrt(((i)**2)+((j)**2)+((k)**2))-9.8)

for i in range(1, len(accel)):
    vel.append(accel[i]*0.007 + vel[i - 1])

plt.scatter(time_values[low_index], vel[low_index])
swing_speed = vel[low_index]*2.237
print('Your swing speed was: ', swing_speed)

club = input('Which club are you hitting? ')
if club == 'driver':
    loft = 9
    smash_factor = 1.41
if club == '3 wood':
    loft = 14.5
    smash_factor = 1.4
if club == '5 wood':
    loft = 17.5
    smash_factor = 1.39
if club == '4 hybrid':
    loft = 22
    smash_factor = 1.38
if club == '3 iron':
    loft = 17.5
    smash_factor = 1.38
if club == '4 iron':
    loft = 20.5
    smash_factor = 1.38
if club == '5 iron':
    loft = 23.5
    smash_factor = 1.38
if club == '6 iron':
    loft = 26.5
    smash_factor = 1.38
if club == '7 iron':
    loft = 30
    smash_factor = 1.38
if club == '8 iron':
    loft = 34.5
    smash_factor = 1.3
if club == '9 iron':
    loft = 39.5
    smash_factor = 1.3
if club == 'pw':
    loft = 44.5
    smash_factor = 1.2
if club == '52':
    loft = 52
    smash_factor = 1.16
if club == '56':
    loft = 56
    smash_factor = 1.15
if club == '60':
    loft = 60
    smash_factor = 1.15

dynamic_loft = loft - attack_angle

# Constants
dt = 0.01
g = 9.81
restitution = 0.13 #higher number here is more bouncy
z_floor = 0
min_bounce_height = 0.02

# Air resistance parameters
air_density = 1.25
drag_coefficient = 0.2
radius = 0.02
cross_sectional_area = np.pi * radius**2
mass = 0.046

# Magnus force parameters
ball_speed = swing_speed*smash_factor
rpm = ball_speed * dynamic_loft * 1.67
print('sidespin: ', np.sin(face_to_path)*rpm)
print('rpm:', rpm)
omega = rpm * 2 * np.pi / 60
if face_to_path > 0:
    spin_axis = np.array([np.cos(face_to_path), 0.0, -np.sin(face_to_path)])
elif face_to_path < 0:
    spin_axis = np.array([np.cos(face_to_path), 0.0, np.sin(face_to_path)])
else:
    spin_axis = np.array([1.0, 0.0, 0.0])
"""
A small slice has a spin axis of [0.9, 0, -0.1], while a small hook has [0.9, 0, 0.1]
"""
print('spin axis:', spin_axis)
tau = 2

#do you want any wind?
wind_speed = np.array([0.0, 0.0, 0.0])

spin_axis = spin_axis / np.linalg.norm(spin_axis)

#getting the ball speed with an assumed smash factor, converting to meters/sec, then getting x y and z from spherical coords

print(f'your assumed ball speed was {ball_speed:.2f} mph')
speed_mps = ball_speed * 0.44704
x_vel = 0
y_vel = speed_mps * np.cos(dynamic_loft)
z_vel = speed_mps * np.sin(dynamic_loft)
print('dynamic loft:', dynamic_loft)

position = np.array([0.0, 0.0, 0.0])
velocity = np.array([x_vel, y_vel, z_vel])


trajectory_x, trajectory_y, trajectory_z = [], [], []
bounce = []
sideways = []

for _ in range(1000):
    rel_velocity = velocity - wind_speed
    speed = np.linalg.norm(rel_velocity)  # Magnitude of velocity
    drag_force = 0.5 * air_density * drag_coefficient * cross_sectional_area * speed**2
    drag_direction = (-rel_velocity / speed) if speed != 0 else np.array([0, 0, 0])
    drag_acceleration = (drag_force / mass) * drag_direction

    magnus_coefficient = 0.5 * air_density * cross_sectional_area * radius * 2
    magnus_force = magnus_coefficient * omega * np.cross(spin_axis, rel_velocity)
    magnus_acceleration = magnus_force / mass

    # Spin decay for ang vel
    omega *= np.exp(-dt / tau)

    # Update velocity with gravity, air resistance, and Magnus force
    velocity += (drag_acceleration + magnus_acceleration - np.array([0, 0, g])) * dt

    position += velocity * dt

    # Check for collision with the floor
    if position[2] <= z_floor:
        position[2] = z_floor  # Correct position to the floor
        velocity[2] = -velocity[2] * restitution  # Reverse and reduce z-velocity
        bounce.append(position[1]) #for finding the first time the ball hits the ground
        sideways.append(position[0]) #for seeing how far offline the ball goes

        # Stop simulation if the bounce height is less than 2 cm
        if abs(velocity[2]) < np.sqrt(2 * g * min_bounce_height):
            break

    trajectory_x.append(position[0] * 1.09361)
    trajectory_y.append(position[1] * 1.09361)
    trajectory_z.append(position[2] * 1.09361)

# Rolling phase
roll_speed = velocity[2]
rolling_distance = roll_speed**2 / (2 * 0.2 * g)
rolling_direction = velocity[:2] / roll_speed if roll_speed != 0 else np.array([0, 0])

rolling_steps = int(rolling_distance / (roll_speed * dt))

for _ in range(rolling_steps):
    position[:2] += rolling_direction * roll_speed * dt
    trajectory_x.append(position[0] * 1.09361)
    trajectory_y.append(position[1] * 1.09361)
    trajectory_z.append(z_floor)

# Convert distances to yards
rolling_distance_x = rolling_direction[0] * rolling_distance * 1.09361
rolling_distance_y = rolling_direction[1] * rolling_distance * 1.09361
total_distance = position[1] * 1.09361 + rolling_distance_y

print(f"The ball hit the ground at {min(bounce) * 1.09361:.2f} yards and stopped bouncing at: {position[1] * 1.09361:.2f} yards")
print(f"The ball went {min(sideways) * 1.09361:.2f} yards offline")
print(f"Final rolling distance in X direction: {rolling_distance_x:.2f} yards")
print(f"Final rolling distance in Y direction: {rolling_distance_y:.2f} yards")
print(f"The total distance of the shot is: {total_distance:.2f} yards")

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

ax.plot(trajectory_x, trajectory_y, trajectory_z, label="Ball Trajectory", color="blue")
ax.scatter(trajectory_x[-1], trajectory_y[-1], trajectory_z[-1], color="red", label="Final Position", s=10)


ax.set_xlabel("X (yds)")
ax.set_ylabel("Y (yds)")
ax.set_zlabel("Z (yds)")
ax.set_title("3D Ball Trajectory")
ax.set_box_aspect([1, 1, 1])
ax.legend()

plt.show()